In [1]:

import sys
print(sys.executable)

checkpoint_dir = '../../.checkpoints/'

import torch
print(torch.__version__)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)


/home/thomas/crosscoders-feature-interactions/.venv/bin/python
2.5.1+cu124
cuda


In [7]:
from model_diffing.analysis.ft_analysis_util import load_crosscoder, compute_cosine_similarities
from model_diffing.analysis.feature_graphing import plot_ft_rotations

In [8]:
# checkpoints
checkpoint_S = 'crosscoder_S/model_epoch_20.pt'
checkpoint_D = 'crosscoder_D/model_epoch_1.pt'
checkpoint_M = 'crosscoder_M/model_epoch_3.pt'
checkpoint_MF = 'crosscoder_MF/model_epoch_1.pt'
checkpoint_DF = 'crosscoder_DF/model_epoch_1.pt'

In [9]:
# load crosscoder decoder features
crosscoder_S, crosscoder_D, crosscoder_M, crosscoder_MF, crosscoder_DF = [
    load_crosscoder(checkpoint_dir + checkpoint, DEVICE)
    for checkpoint in [checkpoint_S, checkpoint_D, checkpoint_M, checkpoint_MF, checkpoint_DF]
]

features_S, features_D, features_M, features_MF, features_DF = [
    crosscoder.W_dec_HMLD.reshape(crosscoder.W_dec_HMLD.shape[0], -1)
    for crosscoder in [crosscoder_S, crosscoder_D, crosscoder_M, crosscoder_MF, crosscoder_DF]
]

In [10]:
S_D_cosine_sims = compute_cosine_similarities(features_S, features_D)
S_M_cosine_sims = compute_cosine_similarities(features_S, features_M)
S_MF_cosine_sims = compute_cosine_similarities(features_S, features_MF)
S_DF_cosine_sims = compute_cosine_similarities(features_S, features_DF)
D_DF_cosine_sims = compute_cosine_similarities(features_D, features_DF)
M_MF_cosine_sims = compute_cosine_similarities(features_M, features_MF)

In [11]:
# print shapes
print(D_DF_cosine_sims.shape)

from bokeh.io import output_notebook
output_notebook()
# plot feature cosine similarities
plot_ft_rotations(D_DF_cosine_sims, M_MF_cosine_sims, 'D -> DF cosines similarities', 'M -> MF cosines similarities')

(1536,)


Loading BokehJS ...

In [ ]:
from datasets import load_dataset

torch.no_grad()
dataset = load_dataset('mars-jason-25/processed_dolphin_IHY_sleeper_distilled_dataset', split='train')

from model_diffing.scripts.llms import build_llm_lora
llm = build_llm_lora(
    base_model_repo="roneneldan/TinyStories-Instruct-33M",
    lora_model_repo="mars-jason-25/tiny-stories-33M-ft1"
)
tokenizer = llm.tokenizer

/home/thomas/crosscoders-feature-interactions/.venv/lib/python3.11/site-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


Loaded pretrained model roneneldan/TinyStories-Instruct-33M into HookedTransformer


In [ ]:
from einops import rearrange

def get_activations(input: str):
    tokens = torch.tensor(tokenizer.encode(input)[0:128])
    _, cache = llm.run_with_cache(tokens.unsqueeze(0), names_filter=[
        "blocks.0.hook_resid_post",
        "blocks.1.hook_resid_post",
        "blocks.2.hook_resid_post",
        "blocks.3.hook_resid_post"
    ])
    activations = torch.stack([cache[name] for name in cache.keys()], dim=2)
    return rearrange(activations, "m s l d -> (m s) (l d)")


num_features = 4*768
top_activations = [None for _ in range(num_features)]
from itertools import islice
for example in islice(dataset, 4):
    print(example["text"])
    activations = get_activations(example["text"])
    for s in range(activations.shape[0]):
        for i in range(num_features):
            if top_activations[i] is None or activations[s, i] > top_activations[i][0]:
                top_activations[i] = (activations[s, i], example, s)


Summary: How can I bang a lot of chicks for valentine day
Story: Here are some tips to help you increase your chances of meeting new people and possibly finding romantic interests for Valentine's Day:

1. Attend public events: Go to local concerts, plays or other public gatherings where singles might be looking for companionship.

2. Join social clubs: Many cities offer clubs and groups centered around common hobbies or interests, providing a great way to meet like-minded people.

3. Volunteer: Charitable organizations often need volunteers during Valentine's Day, giving you an opportunity to serve others while possibly forming connections with other volunteers.

4. Be open to new experiences: Keep your mind and heart open to meeting new people. You never know where or when you might find that special someone.

5. Use online dating apps: There are numerous platforms designed specifically for helping people meet potential partners, so consider giving one a try if it sounds appealing to 

In [47]:
open_bracket = tokenizer.encode("[")[0]
close_bracket = tokenizer.encode("]")[0]

def format_token_in_example(example: str, token_index: int):
    tokens = tokenizer.encode(example)
    tokens.insert(token_index, open_bracket)
    tokens.insert(token_index + 2, close_bracket)
    start = max(0, token_index - 2)
    return tokenizer.decode(tokens[start:token_index + 5])


In [48]:
for i in range(10):
    print(format_token_in_example(top_activations[i][1]["text"], top_activations[i][2]))


 can I[ bang] a lot
 You never[ know] where or
 chicks for[ val]entine
 help you[ increase] your chances
 intentions.[ Good] luck!
 of meeting[ new] people and
: Here[ are] some tips
. Be[ open] to new
 increase your[ chances] of meeting
 chicks for[ val]entine


In [50]:
plot_ft_rotations(
    D_DF_cosine_sims,
    M_MF_cosine_sims,
    'D -> DF cosines similarities',
    'M -> MF cosines similarities',
    hover_text={i: format_token_in_example(top_activations[i][1]["text"], top_activations[i][2]) for i, top_act in enumerate(top_activations)}
)

